<a href="https://colab.research.google.com/github/DhvanilSanghvi/advanced-algo-trading/blob/master/bayesian-generalized-linear-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
sns.set(style="darkgrid", palette="muted")

def simulate_linear_data(N, beta_0, beta_1, eps_sigma_sq):
  df = pd.DataFrame(
    {"x":
      np.random.RandomState(42).choice(
        map(
          lambda x: float(x)/100.0,
          np.arange(100)
        ), N, replace=False
      )
    }
  )
  eps_mean = 0.0
  df["y"] = beta_0 + beta_1*df["x"] + np.random.RandomState(42).normal(
    eps_mean, eps_sigma_sq, N
  )
  return df

if __name__ == "__main__":
  beta_0 = 1.0
  beta_1 = 2.0
  N = 100
  eps_sigma_sq = 0.5
  df = simulate_linear_data(N, beta_0, beta_1, eps_sigma_sq)
  sns.lmplot(x="x", y="y", data=df, size=10)
  plt.xlim(0.0, 1.0)

In [ ]:
import pymc3 as pm

def glm_mcmc_inference(df, iterations=5000):
  """
  Calculates the Markov Chain Monte Carlo trace of
  a Generalised Linear Model Bayesian linear regression
  model on supplied data.
  df: DataFrame containing the data
  iterations: Number of iterations to carry out MCMC for
  """
  baisc_model = pm.Model()
  with basic_model:
    # Create the glm using the Patsy model syntax
    # We use a Normal distribution for the likelihood
    pm.glm.glm("y ~ x", df, family=pm.glm.families.Normal())
    # Use Maximum A Posteriori (MAP) optimisation
    # as initial value for MCMC
    start = pm.find_MAP()
    step = pm.NUTS()
    trace = pm.sample(
        iterations, step, start,
        random_seed=42, progressbar = True
    )

    return trace

if __name__ == "__main__":
  beta_0 = 1.0
  beta_1 = 2.0
  N = 100
  eps_sigma_sq = 0.5
  df = simulate_linear_data(N, beta_0, beta_1, eps_sigma_sq)
  sns.lmplot(x="x", y="y", data=df, size=10)
  plt.xlim(0.0, 1.0)
  trace = glm_mcmc_inference(df, iterations=5000)
  pm.traceplot(trace[500:])
  plt.show()
  # Plot a sample of posterior regression lines
  sns.lmplot(x="x", y="y", data=df, size=10, fit_reg=False)
  plt.xlim(0.0, 1.0)
  plt.ylim(0.0, 4.0)
  pm.glm.plot_posterior_predictive(trace, samples=100)
  x = np.linspace(0, 1, N)
  y = beta_0 + beta_1*x
  plt.plot(x, y, label="True Regression Line", lw=3., c="green")
  plt.legend(loc=0)
  plt.show()
